## Simple RAG Application

### Document Loader

In [106]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
urls = ["https://transformer-circuits.pub/2025/attribution-graphs/biology.html","https://transformer-circuits.pub/2025/attribution-graphs/biology.html#introduction"]
loader = WebBaseLoader(urls)
text_document = loader.load()
text_document[0]

Document(metadata={'source': 'https://transformer-circuits.pub/2025/attribution-graphs/biology.html', 'title': 'On the Biology of a Large Language Model', 'language': 'en'}, page_content='\n\n\n\nOn the Biology of a Large Language Model\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n×\n\n\n\n\n\n\n\n\n\n\n\nTransformer Circuits Thread\n\n\nOn the Biology of a Large Language Model\nOn the Biology of a Large Language ModelWe investigate the internal mechanisms used by Claude 3.5 Haiku — Anthropic\'s lightweight production model — in a variety of contexts, using our circuit tracing methodology.Introductory Example: Multi-step ReasoningPlanning in PoemsMultilingual CircuitsAdditionMedical DiagnosesEntity Recognition and HallucinationsRefusalsLife of a JailbreakChain-of-thought FaithfulnessUncovering Hidden Goals in a Misaligned ModelCommonly Observed Circuit Components and StructureLimitations   Authors   Jack Lindsey†,Wes Gurnee*,Emmanuel Amei

### Loading API Keys

In [107]:
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


### Splitting the Document Text

In [108]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=50)
splitted_document = text_splitter.split_documents(text_document)
splitted_document[:5]

[Document(metadata={'source': 'https://transformer-circuits.pub/2025/attribution-graphs/biology.html', 'title': 'On the Biology of a Large Language Model', 'language': 'en'}, page_content='On the Biology of a Large Language Model\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n×\n\n\n\n\n\n\n\n\n\n\n\nTransformer Circuits Thread'),
 Document(metadata={'source': 'https://transformer-circuits.pub/2025/attribution-graphs/biology.html', 'title': 'On the Biology of a Large Language Model', 'language': 'en'}, page_content='On the Biology of a Large Language Model'),
 Document(metadata={'source': 'https://transformer-circuits.pub/2025/attribution-graphs/biology.html', 'title': 'On the Biology of a Large Language Model', 'language': 'en'}, page_content="On the Biology of a Large Language ModelWe investigate the internal mechanisms used by Claude 3.5 Haiku — Anthropic's lightweight production model — in a variety of contexts, using our circuit"),
 Do

### Embedding the Document in FAISS DB

In [109]:
from langchain_community.vectorstores import FAISS
embedder = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(splitted_document,embedder)

### Querying Vector Database

In [110]:
query = "Who mentioned that AI is the new-electricity ?"
result = db.similarity_search(query)
# result[1].page_content
list(db.docstore._dict.values())

[Document(id='5c6840cc-241d-49b9-9e28-c3b294f2cce8', metadata={'source': 'https://transformer-circuits.pub/2025/attribution-graphs/biology.html', 'title': 'On the Biology of a Large Language Model', 'language': 'en'}, page_content='On the Biology of a Large Language Model\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n×\n\n\n\n\n\n\n\n\n\n\n\nTransformer Circuits Thread'),
 Document(id='c7538bac-0ad9-4590-814a-df9e6a2a1170', metadata={'source': 'https://transformer-circuits.pub/2025/attribution-graphs/biology.html', 'title': 'On the Biology of a Large Language Model', 'language': 'en'}, page_content='On the Biology of a Large Language Model'),
 Document(id='82ef52e7-a378-44ac-a650-f295a0238606', metadata={'source': 'https://transformer-circuits.pub/2025/attribution-graphs/biology.html', 'title': 'On the Biology of a Large Language Model', 'language': 'en'}, page_content="On the Biology of a Large Language ModelWe investigate the internal me

In [111]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [112]:
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3
)

In [115]:
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
rag_chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = retriever,
    memory = memory
)
while True:
    query = input()
    if query.lower == "quit":
        break
    response = rag_chain.invoke(query)
    print(response['answer'])

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


KeyboardInterrupt: 